## Faster parsing of the Nalu acquisition format using Rust

### Naludaq Version
*Min Version*: `0.17.2`
### Naluacq Version
*Min version*: `0.4.1`

In [ ]:
%pip show naluacq

In [ ]:
%pip show naludaq

### Prerequisites
+ Acquisition captured previously (can be from NaluScope)

In [ ]:
# Run this code for plotting
import matplotlib
import matplotlib.pyplot as plt


def plot_event(event: dict, channels: list[int] = None, title: str = None):
    """Plot an event dict.

    Args:
        event (dict): the parsed event
        channels (list[int]): Channels to plot. Defaults to all channels.
        title (str): Optional title for the plot
    """
    channels = channels or range(len(event["data"]))
    title = title or f"Event #{event.get('pkg_num', '?')}"

    # Plot the event
    plt.figure(figsize=(8, 8), constrained_layout=True)
    plt.xlabel("Time")
    plt.ylabel("ADC Counts")
    plt.title(title)
    cmap = get_color_mapping("ocean")
    set_plot_style(font_size=18, font_family="monospace")
    for i, channel in enumerate(channels):
        time = event["time"][channel]
        data = event["data"][channel]
        plt.plot(
            time, data, ".-", label=f"Channel {channel}", color=cmap(i / len(channels))
        )
    plt.legend()
    plt.show()


def set_plot_style(font_size: int = 18, font_family: str = "monospace"):
    """Sets the matplotlib font size and family"""
    matplotlib.rcParams.update({"font.size": font_size})
    matplotlib.rcParams.update({"font.family": font_family})


def get_color_mapping(name: str = "ocean"):
    """Gets a matplot lib color map"""
    cmap = matplotlib.cm.get_cmap(name)
    return cmap


def simple_event_plot(
    event: dict,
    title: "str|None" = None,
    xlabel: "str|None" = None,
    ylabel: "str|None" = None,
    legend: bool = True,
    cmap=None,
):
    """Plot a single event in a simple line graph.

    Args:
        event (dict): the event to plot. Must be parsed.
        title (str): the title of the plot.
        legend (bool): whether to show the legend.
        cmap: the color map to use for the lines.
    """
    plt.title(title or f"Event {event['event_num']}")
    plt.xlabel(xlabel or "Sample Number")
    plt.ylabel(ylabel or "ADC Value (counts)")
    for channel, data in enumerate(event["data"]):
        extra_kwargs = {}
        if cmap:
            extra_kwargs["color"] = cmap(channel / len(event["data"]))
        plt.plot(data, label=f"Channel {channel}", **extra_kwargs)
    if legend:
        plt.legend()
    plt.show()

## Acquisition Format

Acquisitions captured using the Nalu Scientific hardware server use a different data format than legacy acquisitions in order to support higher readout rates and add some failsafe mechanisms

```
acquisition/
├── metadata.yml - Stores board parameters and register values at time of capture
├── {n}.bin - One or more "chunks" containing raw events stored back-to-back in chronological order. See below for more information
├── {n}.idx - Contains information about where in the `.bin` files each event is located. See below for more information
├── (pedestals_calibration) - Pedestals calibration data, stored as a Python pickle
├── (timing_calibration) - Timing calibration data, stored as a Python pickle
└── (adc2mv_calibration) - ADC/mV calibration data, stored as a Python pickle
```

### Bin File Format
Each `.bin` file is referred to as a "chunk." Each chunk is limited to 500 MB and begins with the following header:

```rust
// Struct is written according to native endianness
struct Header {
    // Format revision number
    version: u16,
    // Reserved for future use
    _reserved: u16,
    // Length of the metadata sector
    metadata_sector_length: u32,
}
```

The metadata sector follows the header, and contains a copy of the `metadata.yml` file.

### Idx File Format
Each `.idx` file is referred to as an "index file," and contains a long list of "entries" holding information about where each event is located in the chunk file with the corresponding name. Each entry has the following format:

```rust
// Struct is written according to native endianness
struct IndexEntry {
    // Offset in bytes of the event in the chunk.
    offset: u32,
    // Length of the event
    length: u32,
}
```

### Opening Acquisitions

This example applies to acquisitions captured using the server. For legacy acquisitions, see the other notebook.

In [ ]:
from naluacq import Acquisition

In [ ]:
ACQ_PATH = r"D:\NaluData\2025-02-06 18-17-31.784795"

In [ ]:
acq = Acquisition(root=ACQ_PATH)
PEDS = acq.pedestals_calibration["data"]
PARAMS = acq.params
print(len(acq))

Events can be accessed in either raw or parsed form. For most purposes, parsed events are desired. Acquisitions are not loaded into memory because they can be very large. As a consequence, this means they currently are loaded from disk each time. If this is a concern, it is recommended to cache the results.

In [ ]:
EVENT_INDEX = 0

# Accessing events with the subscript operator will parse the events automatically.
# The subscript operator doesn't supports slices.
EVENT = acq[EVENT_INDEX]

# Missing data field means the event could not be parsed
if EVENT.get("data", None) is None:
    print("Event is corrupted!")
else:
    plot_event(EVENT)

## Pedestals Subtraction

For both acquisition formats, the events stored in the acquisitions are not pedestal-subtracted. To obtain pedestals-subtracted events, the process must be performed manually on the events.

In [ ]:
from naludaq.tools.pedestals.pedestals_correcter import PedestalsCorrecter

# Fetch an event, pedestals, and params from disk
ACQ_PATH = r"<PATH TO ACQUISITION FOLDER>"
EVENT_INDEX = 0
acq = Acquisition(ACQ_PATH)
event = acq[EVENT_INDEX]
params = acq.params
pedestals = acq.pedestals_calibration

if pedestals is None:
    print("Acquisition must have pedestals in order to correct data!")
else:
    # Apply correction (in place)
    corrector = PedestalsCorrecter(params, pedestals)
    corrected_event = corrector.run(event, correct_in_place=True)
    plot_event(corrected_event)

## Process multiple events in a loop

In [ ]:
from tqdm import tqdm

In [ ]:
from naludaq.tools.pedestals.pedestals_correcter import PedestalsCorrecter

# Fetch an event, pedestals, and params from disk
ACQ_PATH = r"<PATH TO ACQUISITION FOLDER>"
EVENT_INDEX = 0
acq = Acquisition(ACQ_PATH)
event = acq[EVENT_INDEX]
params = acq.params
pedestals = acq.pedestals_calibration
corrector = PedestalsCorrecter(params, pedestals)
EVENTS = []
for i in tqdm(range(len(acq))):
    # Catch bad events, getting parsed events can fail and the iterator stops unexpected.
    # Use indexes instead.
    try:
        evt = acq[i]
    except Exception:
        print(f"Event {i}, can't be opened")
    # Apply pedestals correction (in place)
    corrected_event = corrector.run(event, correct_in_place=True)

    #
    # Process the pedestals corrected event here
    #

    # Store the processed data
    EVENTS.append(corrected_event)